# <center>Lab 2: NN (Fixed TensorFlow 2.x problem)  <center>

### Import Libaries

In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
from matplotlib.pyplot import rcParams
import seaborn as sn
%matplotlib inline

### Import titanic dataset

In [2]:
url = 'https://raw.githubusercontent.com/korpong-si/propk/master/titanic.csv'

In [3]:
df = pd.read_csv(url)
df_dummy = pd.get_dummies(df)
df_dummy.head()

,Survived,Pclass,Age,SibSp,Parch,FamilySize,Fare,Cabin,Title_Master,Title_Miss,Title_Mr,Title_Mrs,Title_Rare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,0,3,22.0,1,0,2,7.2500,0,0,0,1,0,0,0,1,0,0,1
1,1,1,38.0,1,0,2,71.2833,1,0,0,0,1,0,1,0,1,0,0
2,1,3,26.0,0,0,1,7.9250,0,0,1,0,0,0,1,0,0,0,1
3,1,1,35.0,1,0,2,53.1000,1,0,0,0,1,0,1,0,0,0,1
4,0,3,35.0,0,0,1,8.0500,0,0,0,1,0,0,0,1,0,0,1


### Split training data and  test data 

In [4]:
from sklearn.model_selection import train_test_split

outcomename = 'Survived'
X = df_dummy.drop(columns = outcomename)
Y = df_dummy[outcomename]

featurename = X.columns
outcome_value = ["0","1"]

X_train, X_test, y_train, y_test = train_test_split( X, Y, test_size = 0.2, random_state = 1234,stratify = Y)
print('training set = {} records, test set= {} records'.format(X_train.shape[0],X_test.shape[0]))

training set = 712 records, test set= 179 records


### StandardScaler (Z-score)


In [5]:
from sklearn.preprocessing import StandardScaler  
scaler = StandardScaler()  
scaler.fit(X_train)

X_train_scal = scaler.transform(X_train)  
X_test_scal = scaler.transform(X_test)

X_train_scal = pd.DataFrame(data=X_train_scal, index=X_train.index, columns=X_train.columns)
X_test_scal = pd.DataFrame(data=X_test_scal, index=X_test.index, columns=X_test.columns)

***
# <center><font color=purple> Neural Network</font></center>

### Import libraries

__Keras__ is a high-level neural networks API, written in Python and capable of running on top of TensorFlow, CNTK, or Theano. It was developed with a focus on enabling fast experimentation.

In [6]:
%tensorflow_version 1.x

UsageError: Line magic function `%tensorflow_version` not found.


In [ ]:
import tensorflow
print(tensorflow.__version__)

In [ ]:
! pip install keras_metrics

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation
import keras_metrics as eval_matrics
from tensorflow import set_random_seed
from numpy.random import seed

### Set random seed 

In [ ]:
seed(1043)
set_random_seed(1043)

### Model config

- __activations__ : softmax, relu, tanh, sigmoid
- __loss functions__ : binary_crossentropy, mean_squared_error
- __optimizer__: SGD, RMSprop, Adam

In [ ]:
input_dims = X_train.shape[1]
output_dims = 1

first_layer_node = 8
second_layer_node = 8

activation_f = 'relu'
activation_output = 'sigmoid'

optimizer = 'Adam'
loss_function = 'binary_crossentropy'
eva_matrics = ['accuracy', eval_matrics.binary_precision(), eval_matrics.binary_recall()]

### Build model

In [ ]:
model = Sequential()
model.add(Dense(first_layer_node, input_dim = input_dims , activation = activation_f))
model.add(Dense(second_layer_node, activation = activation_f))
model.add(Dense(output_dims, activation = activation_output))

model.compile(loss= loss_function, optimizer = optimizer, metrics= eva_matrics) 
model.summary()

### Train Model

- __Epoch__ : a single step in training a neural network; One Epoch is when an entire dataset is passed forward through the neural network.
- __Batch Size__ : Total number of training examples pass into the neural net at once.
- __Verbose__ : Show the training progress for each epoch. (0 : nothing, 1: show process with animated progress bar , 2 :show process without animated progress bar 



In [ ]:
epochs = 200
batch_size = 10 
nn = model.fit(X_train, y_train, validation_data=(X_test,y_test), epochs=epochs, batch_size=batch_size, verbose=1)

### Prediction 

In [ ]:
nn_pred_train = np.ravel(model.predict_classes(X_train))
nn_pred_test = np.ravel(model.predict_classes(X_test))

print('Target classes of the first five rows are {}'.format(y_test.values[0:5]))
print('Predicted classes of the first five rows are {}'.format(nn_pred_test[0:5]))

### Evaluation 

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, balanced_accuracy_score, classification_report, precision_score, recall_score

In [ ]:
def cm_plot(cm):
    plt.clf()
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.coolwarm)
    classNames = ['Not Survived','Survived']
    plt.title('Confusion Matrix')
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    tick_marks = np.arange(len(classNames))
    plt.xticks(tick_marks, classNames, rotation=45)
    plt.yticks(tick_marks, classNames)
    s = [['TN','FP'], ['FN', 'TP']]
    plt.grid(None)
    plt.rcParams.update({'font.size': 10})
    plt.rcParams["font.weight"] = "bold"
    for i in range(2):
        for j in range(2):
            plt.text(j,i, str(s[i][j])+" = "+str(cm[i][j]))
    plt.show()

In [ ]:
def plot_metrics(nn,metric):
    plt.figure(figsize=(12, 6))  
    plt.rcParams.update({'font.size': 12})
    plt.plot(nn.history[metric])
    plt.plot(nn.history['val_'+ metric])
    plt.title('Model '+ metric)
    plt.ylabel(metric)
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='lower right')
    plt.show()

In [ ]:
plot_metrics(nn,'acc')

In [ ]:
plot_metrics(nn,'precision')

In [ ]:
plot_metrics(nn,'recall')

In [ ]:
test_acc = accuracy_score(y_test,nn_pred_test)*100
test_bl_acc = balanced_accuracy_score(y_test,nn_pred_test)*100
test_cm = confusion_matrix(y_test,nn_pred_test)
test_report = classification_report(y_test,nn_pred_test)

print("Test set | Accuracy is", test_acc)
print("Test set | Balance Accuracy is", test_bl_acc)
print("")
cm_plot(test_cm)
print(test_report)

### **Test NN On Scaled Data**

In [ ]:
nn_scal = model.fit(X_train_scal, y_train, validation_data=(X_test_scal,y_test), epochs=200, batch_size=batch_size, verbose=1)

In [ ]:
plot_metrics(nn_scal,'acc')

In [ ]:
plot_metrics(nn_scal,'precision')

In [ ]:
plot_metrics(nn_scal,'recall')

In [ ]:
nn_pred_train = np.ravel(model.predict_classes(X_train_scal))
nn_pred_test = np.ravel(model.predict_classes(X_test_scal))

print('Target classes of the first five rows are {}'.format(y_test.values[0:5]))
print('Predicted classes of the first five rows are {}'.format(nn_pred_test[0:5]))

test_acc = accuracy_score(y_test,nn_pred_test)*100
test_bl_acc = balanced_accuracy_score(y_test,nn_pred_test)*100
test_cm = confusion_matrix(y_test,nn_pred_test)
test_report = classification_report(y_test,nn_pred_test)

print("Test set | Accuracy is", test_acc)
print("Test set | Balance Accuracy is", test_bl_acc)
print("")
cm_plot(test_cm)
print(test_report)

### Test NN on Scal Data with Separate Topology

In [ ]:
input_dims = X_train.shape[1]
output_dims = 1

first_layer_node = 8
second_layer_node = 8

activation_f = 'relu'
activation_output = 'sigmoid'

optimizer = 'Adam'
loss_function = 'binary_crossentropy'
eva_matrics = ['accuracy', eval_matrics.binary_precision(), eval_matrics.binary_recall()]

In [ ]:
model = Sequential()
model.add(Dense(first_layer_node, input_dim = input_dims , activation = activation_f))
model.add(Dense(second_layer_node, activation = activation_f))
model.add(Dense(output_dims, activation = activation_output))

model.compile(loss= loss_function, optimizer = optimizer, metrics= eva_matrics) 
model.summary()

In [ ]:
nn_scal = model.fit(X_train_scal, y_train, validation_data=(X_test_scal,y_test), epochs=20, batch_size=batch_size, verbose=1)

In [ ]:
plot_metrics(nn_scal,'acc')

In [ ]:
plot_metrics(nn_scal,'precision')

In [ ]:
plot_metrics(nn_scal,'recall')

In [ ]:
nn_pred_train = np.ravel(model.predict_classes(X_train_scal))
nn_pred_test = np.ravel(model.predict_classes(X_test_scal))

print('Target classes of the first five rows are {}'.format(y_test.values[0:5]))
print('Predicted classes of the first five rows are {}'.format(nn_pred_test[0:5]))

test_acc = accuracy_score(y_test,nn_pred_test)*100
test_bl_acc = balanced_accuracy_score(y_test,nn_pred_test)*100
test_cm = confusion_matrix(y_test,nn_pred_test)
test_report = classification_report(y_test,nn_pred_test)

print("Test set | Accuracy is", test_acc)
print("Test set | Balance Accuracy is", test_bl_acc)
print("")
cm_plot(test_cm)
print(test_report)